In [ ]:
import torch
device = torch.device("mps")

import evaluate
import sacrebleu
from tqdm import tqdm

from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

### Part 1) Preparing and tokenizing the training datasets


In [36]:
# function to prepare the training and any other dataset
def prepare_data(eng_file_path, mai_file_path, data_type):
    # load the english and maithili texts files
    with open(eng_file_path, "r", encoding="utf-8") as en_file:
        eng_texts = en_file.readlines()

    with open(mai_file_path, "r", encoding="utf-8") as maithili_file:
        mai_texts = maithili_file.readlines()

    assert len(eng_texts) == len(mai_texts), "The number of sentences in both files must be the same."

    # clean the text files
    eng_texts_cleaned = [text.strip() for text in eng_texts]
    mai_texts_cleaned = [text.strip() for text in mai_texts]

    # create the dataset
    data = {
        "source_text": eng_texts_cleaned,
        "target_text": mai_texts_cleaned, 
    }
    dataset = Dataset.from_dict(data)

    # split the dataset into train, validation and test sets
    if data_type == "train":
        train_dataset, temp_dataset = dataset.train_test_split(test_size=0.1).values()
        val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5).values()

        print(f"Training set size: {len(train_dataset)}")
        print(f"Validation set size: {len(val_dataset)}")
        print(f"Test set size: {len(test_dataset)}")

        return train_dataset, val_dataset, test_dataset
    elif data_type == "test":
        return dataset

# preprocessor function for tokenizer
def preprocess_function(examples, tokenizer):
    inputs = tokenizer(examples["source_text"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(examples["target_text"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

# function to tokenize the data
def tokenize_dataset(dataset, tokenizer):
    dataset_tokenized = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    return dataset_tokenized

# function to move the dataset to device
def move_to_device(batch):
    # move each tensor in the batch to the MPS device
    for key in batch:
        batch[key] = torch.tensor(batch[key]).to(device)
    return batch


In [37]:
# prepare the data
train_dataset, val_dataset, test_dataset = prepare_data("./dataset/train/bpcc/train.eng_Latn", "./dataset/train/bpcc/train.mai_Deva", "train")

# tokenize the data
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi") # load the tokenizer
train_dataset_tokenized = tokenize_dataset(train_dataset, tokenizer)
val_dataset_tokenized = tokenize_dataset(val_dataset, tokenizer)
test_dataset_tokenized = tokenize_dataset(test_dataset, tokenizer)

# # saving dataset to csv (backup)
# train_dataset_tokenized.to_csv("./dataset/training/bpcc/train_dataset.csv")
# val_dataset_tokenized.to_csv("./dataset/training/bpcc/val_dataset.csv")
# test_dataset_tokenized.to_csv("./dataset/training/bpcc/test_dataset.csv")


Training set size: 60892
Validation set size: 3383
Test set size: 3383


Map:   0%|          | 0/60892 [00:00<?, ? examples/s]

Map:   0%|          | 0/3383 [00:00<?, ? examples/s]

Map:   0%|          | 0/3383 [00:00<?, ? examples/s]

In [38]:
# checking the tokenization and vocab subwords
print("Source text: ", train_dataset_tokenized[0]["source_text"])
print("Target text: ", train_dataset_tokenized[0]["target_text"])
print("Source tokens: ", tokenizer.convert_ids_to_tokens(train_dataset_tokenized[0]["input_ids"]))
print("Target tokens: ", tokenizer.convert_ids_to_tokens(train_dataset_tokenized[0]["labels"]))

# save the tokenized output to a text file
with open("dataset/training/tokenized_output.txt", "w", encoding="utf-8") as f:
    f.write(train_dataset_tokenized[0]["source_text"] + "\n")
    f.write(train_dataset_tokenized[0]["target_text"] + "\n")
    f.write(" ".join(tokenizer.convert_ids_to_tokens(train_dataset_tokenized[0]["input_ids"])) + "\n")
    f.write(" ".join(tokenizer.convert_ids_to_tokens(train_dataset_tokenized[0]["labels"])) + "\n")

Source text:  The Flyers ended up winning the game 5-3, adding yet another loss to an already disappointing Bruins season.
Target text:  फ्लायर्स मैच केँ ५-३ सँ जीति कऽ ब्रून्सक पहिनहिसँ निराशाजनक सत्र में एक टा आर पराजय जोड़लनि।
Source tokens:  ['▁The', '▁F', 'ly', 'ers', '▁ended', '▁up', '▁winning', '▁the', '▁game', '▁5', '-3', ',', '▁adding', '▁yet', '▁another', '▁loss', '▁to', '▁an', '▁already', '▁disappointing', '▁Bru', 'in', 's', '▁season', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

### Part 2) Loading the pretrained model and testing its performance on benchmark datasets


In [56]:
# load the model
model_name = "Helsinki-NLP/opus-mt-en-hi"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("Parameter Size (pretrained): ", model.num_parameters())

# prepare the in22 benchmark datasets
in22_hin_test = prepare_data("./dataset/test/IN22_test/gen/test.eng_Latn", "./dataset/test/IN22_test/gen/test.hin_Deva", "test")
in22_mai_test = prepare_data("./dataset/test/IN22_test/gen/test.eng_Latn", "./dataset//test/IN22_test/gen/test.mai_Deva", "test")

print("English Text: ", in22_hin_test[0]["source_text"])
print("Hindi Text: ", in22_hin_test[0]["target_text"])
print("Maithili Text: ", in22_mai_test[0]["target_text"])

# tokenize the in22 benchmark datasets
in22_hin_test_tokenized = tokenize_dataset(in22_hin_test, tokenizer)
in22_mai_test_tokenized = tokenize_dataset(in22_mai_test, tokenizer)


Parameter Size (pretrained):  76381184
English Text:  An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
Hindi Text:  सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।
Maithili Text:  रूप सर्विसवला व्यक्तिसँ सम्बन्धित बहुत रास लक्षणक समूह होयत छै जेना हुनक जूता, कपड़ा, टाई, गहना, केश, श्रृंगार, घड़ी, प्रसाधन सामग्री, सेंट इत्यादि।


Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

In [54]:
#testing the pretrained model on the in22 benchmark datasets

# generate predictions for english to hindi
def generate_predictions(model, tokenizer, test_dataset):
    predictions = []
    references = []
    
    model.eval()  # set model to evaluation mode
    for example in tqdm(test_dataset, desc="Generating predictions: ", unit="example"):
        inputs = torch.tensor(example['input_ids']).unsqueeze(0).to(model.device)  # move input to device
        attention_mask = torch.tensor(example['attention_mask']).unsqueeze(0).to(model.device) # move attention mask to device
        
        with torch.no_grad():
            output = model.generate(inputs, attention_mask=attention_mask, max_length=128) # generate the output
        
        prediction = tokenizer.decode(output[0], skip_special_tokens=True) # decode the output
        target = example['target_text']  # if the target is already a string
        
        predictions.append(prediction)
        references.append(target)
    
    return predictions, references

# define trainer for evaluation
def trainer_evaluate(model, tokenizer, test_dataset):

    eval_trainer = Seq2SeqTrainer(
        model=model,
        args = Seq2SeqTrainingArguments(
            output_dir="./results/test/",
            per_device_eval_batch_size=32,
            predict_with_generate=True,
        ), 
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
    )

    return eval_trainer

def compute_chrf(predictions, references):
    chrf = evaluate.load("chrf")
    chrf_score = chrf.compute(predictions=predictions, references=references)
    return chrf_score


In [ ]:
# move the model and in22 testdata to the MPS device
model = model.to(device)
in22_hin_test_tokenized = in22_hin_test_tokenized.with_transform(move_to_device)
eval_trainer = trainer_evaluate(model, tokenizer, in22_hin_test_tokenized)
hin_pred, hin_lab, _ = eval_trainer.predict(in22_hin_test_tokenized)

hin_pred = tokenizer.batch_decode(hin_pred, skip_special_tokens=True)
hin_ref = tokenizer.batch_decode(hin_lab, skip_special_tokens=True)

  0%|          | 0/32 [00:00<?, ?it/s]

In [57]:
# hin_pred = tokenizer.batch_decode(hin_pred, skip_special_tokens=True)
# hin_ref = tokenizer.batch_decode(hin_lab, skip_special_tokens=True)

for i in range(2):
    print("English Text: ", in22_hin_test[i]["source_text"])
    print("Hindi Reference: ", hin_pred[i])
    print("Hindi Prediction: ", hin_ref[i])
    print("\n")

# calculate chrF++ score for english to hindi translation
chrf_score_hin = compute_chrf(hin_pred, hin_ref)
print(f"chrF++ score for English-Hindi translation: {chrf_score_hin['score']}")

# calculate chrF++ score for hindi to maithili overlap
mai_ref = in22_mai_test["target_text"]
chrf_score_mai = compute_chrf(hin_ref, mai_ref)
print(f"chrF++ score for Hindi-Maithili overlap: {chrf_score_mai['score']}")


English Text:  An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
Hindi Reference:  एक फैशन सेवा व्यक्ति से संबंधित गुणों का एक गुच्छा है, उनके जूते, कपड़े, टाई, बाल शैली, बनाएँ- अप, उपभोग, इत्र, इत्र, इत्यादि.
Hindi Prediction:  सेवा संबंधी लोगों के लिए  कई गों का संयोजन है, जैसे कि उनके , कपड़े, टाई, आ,  शैली, मेक-अप, ़ी, कॉस्मेटिक, इत्र, आ


English Text:  Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
Hindi Reference:  पू. पहली सदी से लेकर सा. यु.
Hindi Prediction:  महाराष्ट्र के औरंगाबाद जिले में स्थित ंता में उन्तीस ्य और ार गुफाएँ हैं जो पहली ाब्दी ई.पू. से ले कर पाीं ाब्दी ्वी तक की मूर


chrF++ score for English-Hindi translation: 28.888480881961343
chrF++ score for Hindi-Maithili overlap: 25.91324614465444


### Part 3) Setting up LoRA and fine tuning the pretrained model


In [58]:
def configure_lora(rank, lora_alpha, lora_dropout, task_type, bias, target_modules):

    lora_config = LoraConfig(
        r=rank,  # rank of the low-rank adaptation
        lora_alpha=lora_alpha,  # scaling factor for the LoRA layers
        lora_dropout=lora_dropout,  # dropout for the LoRA layers
        task_type=task_type, # task type
        bias=bias,  # set bias as 'none', 'all', or 'lora_only'
        target_modules=target_modules  # specify the target modules
    )

    return lora_config

def train_args(output_dir, eval_stra, learning_r, batch_size, grad_step, num_train_epochs, 
            save_steps, logging_dir, logging_steps, save_total_limit):

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,  # directory to save results
        eval_strategy=eval_stra,  # strategy for evaluation
        learning_rate=learning_r,  # learnin rate for fine-tuning
        per_device_train_batch_size=batch_size,  # batch size
        gradient_accumulation_steps= grad_step, # step for backpropagation
        num_train_epochs=num_train_epochs,  # number of training epochs
        save_steps=save_steps,  # save checkpoints after this many steps
        logging_dir=logging_dir,  # directory for logs
        logging_steps=logging_steps, # log after this many steps
        save_total_limit=save_total_limit,  # limit number of saved checkpoints
    )

    return training_args

def trainer_train(model, training_args, train_dataset, val_dataset, tokenizer):

    trainer = Seq2SeqTrainer(
        model=model,  # model to be fine-tuned
        args=training_args,  # training arguments
        train_dataset=train_dataset,  # tokenized training dataset
        eval_dataset=val_dataset, # tokenized validation dataset
        tokenizer=tokenizer  # tokenizer for tokenization
    )

    return trainer

In [ ]:
# configure LoRA
print(f"Before adding LoRA, Parameter Size: {model.num_parameters()}")

lora_config = configure_lora(8, 16, 0.1, "SEQ_2_SEQ_LM", "none", ["q_proj", "v_proj"])
model = get_peft_model(model, lora_config)
print(f"After adding LoRA, Parameter Size: {model.num_parameters()}")

# define training arguments and trainer
training_args = train_args("./results/train", "epoch", 5e-5, 16, 2, 1, 1000, "./logs", 500, 2)
train_trainer = trainer_train(model, training_args, train_dataset_tokenized, val_dataset_tokenized, tokenizer)